In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
import math


# === Device Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load SEED EEG Features ===
DATA_PATH = "D:/SEED_IV/SEED_IV/eeg_feature_smooth"

def pad_or_truncate(features, target_length):
    current_length = len(features)
    if current_length < target_length:
        return np.pad(features, (0, target_length - current_length), mode='constant')
    else:
        return features[:target_length]

def load_eeg_features(sessions=["1", "2", "3"], feature_type="de_movingAve", num_channels=62):
    data = []
    labels = []
    max_length = 0

    for session in sessions:
        session_path = os.path.join(DATA_PATH, session)
        if not os.path.exists(session_path):
            continue

        for file in os.listdir(session_path):
            if not file.endswith(".mat"):
                continue

            file_path = os.path.join(session_path, file)
            mat = scipy.io.loadmat(file_path)

            for trial in range(1, 25):
                key = f"{feature_type}{trial}"
                if key not in mat:
                    continue

                feature_matrix = mat[key]
                if feature_matrix.shape[0] != num_channels:
                    continue

                trial_features = feature_matrix.reshape(-1)
                max_length = max(max_length, len(trial_features))
                data.append(trial_features)
                labels.append((trial - 1) % 3)

    if not data:
        return np.array([]), np.array([])

    data_fixed = np.array([pad_or_truncate(x, max_length) for x in data])
    return np.array(data_fixed), np.array(labels)

# === Load & Normalize Data ===
X_fixed, y = load_eeg_features()
scaler = StandardScaler()
X_fixed = scaler.fit_transform(X_fixed)

X_tensor = torch.tensor(X_fixed, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# === Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EEGNetOptimized(nn.Module):
    def __init__(self, num_classes=3, input_channels=62, input_samples=320):
        super(EEGNetOptimized, self).__init__()

        self.temporal_conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(1, 64), padding=(0, 32), bias=False),
            nn.BatchNorm2d(16),
            nn.ELU(),
            nn.Dropout(0.25)
        )

        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(input_channels, 1), groups=16, bias=False),
            nn.BatchNorm2d(32),
            nn.ELU(),
            nn.Dropout(0.25)
        )

        self.separable_conv = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(1, 16), padding=(0, 8), bias=False),
            nn.BatchNorm2d(64),
            nn.ELU(),
            nn.Dropout(0.25)
        )

        # Drop the classifier here; we'll create it dynamically in forward
        self.fc1 = nn.Linear(1, 1)  # dummy init, will reset later

        self.num_classes = num_classes

    def forward(self, x):
        x = x.unsqueeze(1)  # [batch, 1, channels, samples]
        x = self.temporal_conv(x)
        x = self.depthwise_conv(x)
        x = self.separable_conv(x)

        x = x.flatten(start_dim=1)  # flatten everything but batch dimension
        if isinstance(self.fc1, nn.Linear) and self.fc1.in_features != x.shape[1]:
            self.fc1 = nn.Sequential(
                nn.Linear(x.shape[1], 128),
                nn.ELU(),
                nn.Dropout(0.3),
                nn.Linear(128, self.num_classes)
            ).to(x.device)

        out = self.fc1(x)
        return out

# Initialize model
model = EEGNetOptimized(num_classes=3, input_channels=62, input_samples=320).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        batch_X = batch_X.view(-1, 62, 320)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')

print("✅ EEGNet-Optimized Training Complete!")

# Evaluation
model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        batch_X = batch_X.view(-1, 62, 320)

        outputs = model(batch_X)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(batch_y.cpu().numpy())

# Metrics
acc = accuracy_score(all_targets, all_preds)
report = classification_report(all_targets, all_preds, digits=4)
print(f"\nAccuracy: {acc:.4f}")
print("\nClassification Report:\n", report)


Epoch [1/50], Loss: 53.6716
Epoch [2/50], Loss: 47.3210
Epoch [3/50], Loss: 43.9547
Epoch [4/50], Loss: 40.7553
Epoch [5/50], Loss: 38.3769
Epoch [6/50], Loss: 35.3811
Epoch [7/50], Loss: 33.7105
Epoch [8/50], Loss: 31.3850
Epoch [9/50], Loss: 30.4630
Epoch [10/50], Loss: 28.6442
Epoch [11/50], Loss: 27.7361
Epoch [12/50], Loss: 25.9716
Epoch [13/50], Loss: 25.4036
Epoch [14/50], Loss: 24.3777
Epoch [15/50], Loss: 23.7611
Epoch [16/50], Loss: 23.2216
Epoch [17/50], Loss: 22.5027
Epoch [18/50], Loss: 21.1208
Epoch [19/50], Loss: 20.3296
Epoch [20/50], Loss: 20.2525
Epoch [21/50], Loss: 19.6542
Epoch [22/50], Loss: 19.0874
Epoch [23/50], Loss: 18.3753
Epoch [24/50], Loss: 18.5413
Epoch [25/50], Loss: 18.1042
Epoch [26/50], Loss: 17.8361
Epoch [27/50], Loss: 17.1664
Epoch [28/50], Loss: 17.8592
Epoch [29/50], Loss: 16.8994
Epoch [30/50], Loss: 16.4595
Epoch [31/50], Loss: 16.1298
Epoch [32/50], Loss: 16.4212
Epoch [33/50], Loss: 15.1837
Epoch [34/50], Loss: 15.5273
Epoch [35/50], Loss: 15

In [3]:
def region_wise_analysis(model, X_tensor, y_tensor, region_channels_dict, total_channels=62, input_samples=320):
    model.eval()
    results = {}

    for region, channels in region_channels_dict.items():
        # Create region-specific inputs
        X_region = X_tensor[:, channels, :]

        # Pad to original 62 channels (with zeros for unused channels)
        padded = torch.zeros((X_region.shape[0], total_channels, input_samples))
        padded[:, channels, :] = X_region  # Fill only region's channels

        # Split into test set (same 80/20 split)
        X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
            padded, y_tensor, test_size=0.2, random_state=42
        )

        X_test_reg = X_test_reg.to(device)
        y_test_reg = y_test_reg.to(device)

        # Run model
        with torch.no_grad():
            outputs = model(X_test_reg)
            preds = torch.argmax(outputs, dim=1)
            acc = accuracy_score(y_test_reg.cpu().numpy(), preds.cpu().numpy())

        results[region] = acc

    return results

# === Define Brain Regions by Channel Indices ===
region_channels = {
    'Frontal': list(range(0, 20)),      # Example indices, adjust as per EEG cap layout
    'Temporal': list(range(20, 30)),
    'Parietal': list(range(30, 45)),
    'Occipital': list(range(45, 52)),
    'Central': list(range(52, 62))
}

# === Run Region-Wise Analysis ===
region_scores = region_wise_analysis(model, X_tensor.view(-1, 62, 320), y_tensor, region_channels)

# === Display Results ===
print("\n🧠 Region-wise Accuracy:")
for region, score in region_scores.items():
    print(f"{region}: {score:.4f}")



🧠 Region-wise Accuracy:
Frontal: 0.5972
Temporal: 0.5648
Parietal: 0.5694
Occipital: 0.4398
Central: 0.3796


In [4]:
from torchinfo import summary

summary(model, input_size=(16, 62, 320))  # 16 is a dummy batch size


Layer (type:depth-idx)                   Output Shape              Param #
EEGNetOptimized                          [16, 3]                   --
├─Sequential: 1-1                        [16, 16, 62, 321]         --
│    └─Conv2d: 2-1                       [16, 16, 62, 321]         1,024
│    └─BatchNorm2d: 2-2                  [16, 16, 62, 321]         32
│    └─ELU: 2-3                          [16, 16, 62, 321]         --
│    └─Dropout: 2-4                      [16, 16, 62, 321]         --
├─Sequential: 1-2                        [16, 32, 1, 321]          --
│    └─Conv2d: 2-5                       [16, 32, 1, 321]          1,984
│    └─BatchNorm2d: 2-6                  [16, 32, 1, 321]          64
│    └─ELU: 2-7                          [16, 32, 1, 321]          --
│    └─Dropout: 2-8                      [16, 32, 1, 321]          --
├─Sequential: 1-3                        [16, 64, 1, 322]          --
│    └─Conv2d: 2-9                       [16, 64, 1, 322]          32,768
│    